In [5]:
import pandas as pd
df1 = pd.read_csv(r"cleaned_house.csv")
df1.head(5)

,Unnamed: 0,index,selftext,num_comments,score,title
0,0,11,realize pretty vague honestly mind long simply...,0,1,Trying to source house track I heard on dubpla...
1,1,21,know long shot looking song month luck hoping ...,5,3,"Does anyone have a copy of Rene Breitbarth - ""..."
2,2,49,post dedicated finding unknown unidentifiable ...,9,4,Welcome to our weekly post for song/track IDs....
3,3,91,kind remix like best d,2,1,"Looking for a really good remix of ""Foreigner ..."
4,4,111,like minimal micro house playlist you perfect ...,0,3,Minimal House / Deep Tech Playlist (Submission...


In [6]:
X=pd.DataFrame({'text':df1['selftext']})
X = X.dropna()
X

,text
0,realize pretty vague honestly mind long simply...
1,know long shot looking song month luck hoping ...
2,post dedicated finding unknown unidentifiable ...
3,kind remix like best d
4,like minimal micro house playlist you perfect ...
...,...
4861,looking house track long drum break conga djem...
4862,trying get year contacted label said would sho...
4863,time year yearly round best music year rule si...
4864,mine in particular order hard omar s heard che...


In [7]:
import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords") #uncomment it when run it for the first time
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text
X['text']=X['text'].apply(lambda x: remove_stopwords(x))


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer(max_df=0.5,min_df=0.01, stop_words='english')
counts=vec.fit_transform(X['text'])
counts=counts.toarray()
count_df1=pd.DataFrame(counts,columns=vec.get_feature_names())
#count_df1=count_df1.drop(['amp','don'],axis=1)
count_df1

,able,absolutely,acid,actually,add,advance,ago,album,amazing,amp,...,went,woman,wondering,word,work,working,world,xb,year,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4283,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,2,0
4284,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2,0
4285,0,0,2,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,3,0


In [9]:
from sklearn.decomposition import NMF

model1=NMF(n_components=4,init="random",random_state=0)
model1.fit(count_df1)
#model1.components_

NMF(init='random', n_components=4, random_state=0)

In [10]:
import numpy as np
def top_words(X, model, component, num_words):
    """
    Extract the top words from the specified component 
    for a topic model trained on data. 
    X: a term-document matrix, assumed to be a pd.DataFrame
    model: a sklearn model with a components_ attribute, e.g. NMF
    component: the desired component, specified as an integer. 
        Must be less than than the total number of components in model
    num_words: the number of words to return.
    """
    orders = np.argsort(model.components_, axis = 1)
    important_words = np.array(X.columns)[orders]
    return important_words[component][-num_words:]

In [11]:
topic1=pd.DataFrame({'Topic 0':top_words(count_df1, model1, 0, 10),
                   'Topic 1':top_words(count_df1, model1, 1, 10),
                   'Topic 2':top_words(count_df1, model1, 2, 10),
                   'Topic 3':top_words(count_df1, model1, 3, 10)})
topic1

,Topic 0,Topic 1,Topic 2,Topic 3
0,love,edit,hope,sound
1,disco,feat,love,really
2,playlist,original,sample,love
3,know,party,life,know
4,like,like,drum,dj
5,deep,remix,time,song
6,genre,mix,feel,like
7,song,love,party,mix
8,music,house,xb,music
9,house,amp,amp,track
